# 本章摘要

## 本章内容

1. 如何定义一个机器学习问题
2. 如何开发一个工作模型
3. 如何在生产环境中部署和维护模型

## 机器学习的通用工作流程

机器学习的通用工作流程大致分为以下3步：
1. **定义任务**。了解问题所属领域和客户需求背后的业务逻辑。收集数据集，理解数据所代表的含义，并选择衡量任务成功的指标。
2. **开发模型**。准备数据，使其可以被机器学习模型处理。选择模型评估方法，并确定一个简单基准（模型应能够超越这个基准）。训练第一个具有泛化能力并且能够过拟合的模型，然后对模型进行正则化并不断调节，直到获得最佳泛化性能。
3. **部署模型**。将工作展示给利益相关者，将模型部署到Web服务器、移动应用程序、网页或嵌入式设备上，监控模型在真实环境中的性能，并开始收集构建下一代模型所需的数据。

# 定义问题

## 定义问题

你的输入数据是什么？你要预测什么？只有拥有可用的训练数据，才能学习预测某件事情。<p>
你面对的是什么类型的机器学习任务？是二分类问题、多分类问题、标量回归问题、向量回归问题，还是多分类、多标签问题？是图像分割问题、排序问题，还是聚类、生成式学习或强化学习等其他问题？举例：
- 图片搜索引擎项目是一项多分类、多标签的分类任务。
- 垃圾信息检测项目是一个二分类任务。如果将“攻击性内容”划为一个单独的类别，则它是一项三分类任务。
- 事实证明，对于音乐推荐引擎来说，矩阵分解（协同过滤）比深度学习的效果更好。
- 信用卡欺诈检测项目是一项二分类任务。
- 点击率预测项目是一项标量回归任务。
- 异常饼干检测项目是一项二分类任务。
- 从卫星图像中寻找新的考古遗址，这是一项图像相似度排序任务。<p>

现有的解决方案是什么？<p>
你是否需要处理一些特殊的限制？<p>

**注意**：
- 假设可以根据输入对目标进行预测。
- 假设现有数据（或后续收集的数据）所包含的信息足以用来学习输入和目标之间的关系。

## 收集数据集

## 理解数据

## 选择衡量成功的指标

要在一个项目上获得成功，必须先给出成功的定义：精度、准确率和召回率、还是客户留存率？衡量成功的指标将指引你在整个项目中的所有技术选择。它应该与你的最终目标保持一致。<p>
- **对于平衡分类问题（每个类别的比例相同）**，精度和**受试者操作特征曲线下面积**（ROC AUC）是两个常用指标。
- **对于类别不平衡的问题、排序问题和多标签分类问题**，可以使用准确率和召回率，也可以使用精度或ROC AUC的加权形式。

# 开发模型

模型开发只是机器学习工作流程中的一个步骤（并不是最难的一步）。<p>
机器学习中最难的步骤是：
- 问题定义
- 数据收集
- 数据标注
- 数据清理

## 准备数据

深度学习模型通常不会直接读取原始数据。数据预处理的目的是让原始数据更适于用神经网络处理，它包括：
- 向量化
- 规划化
- 处理缺失值 <p>

许多预处理方法是和特征领域相关的。

### 向量化

神经网络的所有输入和目标通常都必须是浮点数张量。无论要处理什么数据，首先都必须将其转换为张量，这一步叫作**数据向量化**。

### 规范化

一般来说，将取值相对较大的数据（比如多位整数，比神经网络权重的初始值大很多）或异质数据（heterogeneous data，比如数据的一个特征在0 ~ 1范围内，另一个特征在100 ~ 200范围内）输入神经网络是不安全的。这么做可能导致较大的梯度更新，进而导致神经网络无法收敛。为了让神经网络的学习变得更容易，输入数据应具有以下特征。<p>
- **取值较小**：通常情况下，大部分取值应该在0 ~ 1范围内。
- **同质性**：所有特征的取值应该大致在同一范围内。 <p>

下面这种更严格的规范化方法也很常见，而且很有用，不过并非必需。<p>
1. 对每个特征分别规范化，使其均值为0。
2. 对每个特征分别规范化，使其标准差为1。

In [ ]:
# 伪代码
x -= x.mean(axis=0)  # 假设x是一个形状为(samples, features)的二维矩阵
x /= x.std(axis=0)

### 处理缺失值

你的数据有时可能会有缺失值。可以完全舍弃这个特征，但不一定非得这么做。<p>
- **如果是分类特征**，则可以创建一个新的类别，表示“此值缺失”。模型会自动学习这个新类别对于目标的含义。
- **如果是数值特征**，应避免输入像“0”这样随意的值，因为它可能会在特征形成的潜在空间中造成不连续性，从而让模型更加难以泛化。相反，可以考虑用数据集中该特征的**均值**或**中位值**来代替缺失值。也可以训练一个模型，给定其他特征的值，预测该特征的值。<p>

**注意**：如果测试数据的分类特征有缺失值，而训练数据中没有缺失值，那么神经网络无法学会忽略缺失值。这种情况下，应该手动生成一些有缺失值的训练样本：将一些训练样本复制多次，然后舍弃测试数据中可能缺失的某些分类特征。

## 选择评估方法

**模型的目的**是实现泛化。模型开发过程中，所做的每一个建模决定都将以**验证指标**为指导，这些指标的作用是**衡量泛化性能**。<p>
**评估方法的目的**是准确估计在实际生产数据上的成功衡量指标（如精度）。<p>
**三种常用的评估方法：**
- **简单的留出验证：** 数据量很大时可以采用。
- **K折交叉验证：** 如果留出验证的样本太少，无法保证可靠性，则应该使用这种方法。
- **重复K折交叉验证：** 如果可用的数据很少，同时模型评估又需要非常准确，则应该使用这种方法。

## 超越基准

在开始研究模型本身时，你的初始目标是获得**统计功效**：开发一个能够超越简单基准的小模型。<p>
这一阶段，三件非常重要的事情：
- **特征工程**。过滤没有信息量的特征（特征选择），利用对问题的了解，开发可能有用的新特征。
- **选择正确的架构预设**。使用什么类型的模型架构？密集连接网络、卷积神经网络、循环神经网络还是Transformer？深度学习是不是完成这个任务的好方法？
- **选择足够好的训练配置**。
   - 损失函数
   - 批量大小
   - 学习率

### 选择正确的损失函数

#### 为模型正确选择最后一层激活函数和损失函数

问题类型|最后一层激活函数|损失函数
--|:--:|--:
二分类问题|sigmoid|binary_crossentropy
多分类、单标签问题|softmax|categorical_crossentropy
多分类、多标签问题|sigmoid|binary_crossentropy

### 扩大模型规模：开发一个过拟合的模型

要知道需要多大的模型，必须先开发一个过拟合的模型。
1. 增加层数；
2. 让每一层变得更大；
3. 训练更多轮数。

### 模型正则化与调节超参数

如果模型具有统计功效，并且能够过拟合。这时你的目标就变成了将泛化性能最大化。<p>
这一步是最费时间的：你需要不断调节模型、训练模型、在验证数据上评估模型、再次调节模型，然后不断重复这一过程，直到模型达到最佳性能。尝试以下做法：
- 尝试不同的架构，增加或减少层数。
- 添加dropout正则化。
- 如果模型很小，则添加L1正则化或L2正则化。
- 尝试不同的超参数（比如每层的单元个数或优化器的学习率），以找到最佳配置。
- （可选）反复进行数据收集或特征工程：
    - 收集并标注更多的数据
    - 开发更好的特征
    - 删除没有信息量的特征 <p>

利用超参数自动调节软件（如**KerasTuner**），可以将大部分工作自动化。

# 部署模型

## 向利益相关者解释你的工作并设定预期

## 部署推断模型

训练好模型后，机器学习项目并没有结束。很少会将与训练过程中完全相同的Python模型对象投入生产环境中。<p>
- **首先**，可能想将模型导出到Python之外：
  - 生产环境可能不支持Python，比如移动应用或嵌入式系统；
  - 应用程序的其余部分不是用Python编写的，那么用Python实现模型可能会导致大量开销。
- **其次**，由于生产模型只用于输出预测结果[这一过程叫作**推断**]，而不是训练，因此还可以执行各种优化，提高模型速度并减少内存占用。

### 将模型部署为REST API

这是将模型转化为产品的常用方式：在服务期或实例上安装TensorFlow，并通过REST API来查询模型预测结果。既可以使用Flask建立你自己的服务应用程序，也可以使用TensorFlow库将模型部署为API，这叫做**TensorFlow Serving**。

### 在设备上部署模型

### 在浏览器上部署模型

### 推断模型优化

可以使用以下两种常用的优化方法：
- **权重剪枝**
- **权重量化**：单精度浮点(float32)权重进行训练。但可以将权重**量化**为8位有符号整数(int8)，这样得到的推断模型大小只有原始模型的四分之一，但精度扔与原始模型相当。

## 监控模型在真实环境中的性能

## 维护模型